In [3]:
import numpy as np
import mmcfilters
import matplotlib
import math

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16,16)

from IPython import display
from os import listdir
import cv2 as cv


from collections import defaultdict
def dfs(node, depth=0):
    nodes_by_depth[depth].append(node)
    for child in node.children:
        dfs(child, depth + 1)

from PrettyPrint import PrettyPrintTree
from colorama import Fore, Back
printTree = PrettyPrintTree(
    lambda node: node.children,
#    lambda node: ("id:" + str(node.id) + " " + str(node.isMaxtree)[0], "level:" + str(node.level) + " (" + str(node.residue) + ")", "|cnps|:" + str(len(node.cnps))  ), 
    lambda node: ("id:" + str(node.id) ), 
    color=Back.BLACK + Fore.WHITE
)

def mapIDs(tos):
    mapIDs = []
    for p in range(num_rows * num_cols):
        mapIDs.append( tos.getSC(p).id )
    return np.array(mapIDs).reshape(num_rows, num_cols)


def plotCountors(pixels, node, num_rows, num_cols):
    contour_img = node.recNode().reshape(num_rows, num_cols) - 128

    for p in node.pixelsOfCC():
        row = p // num_cols
        col = p % num_cols
        contour_img[row, col] = 128  
        
    for p in pixels:
        row = p // num_cols
        col = p % num_cols
        contour_img[row, col] = 255  
        if(row==8 and col==8):
            contour_img[row, col] = 200

    
    # Plotar com matplotlib
    plt.figure(figsize=(5, 5))
    plt.imshow(contour_img, cmap='Reds', vmax=255, vmin=0)
    plt.title(node)
    plt.axis('off')
    plt.show()

def plotImageBin(nodes_by_depth, num_rows, num_cols):
    numImagens = len(nodes_by_depth)
    fig, axes = plt.subplots(numImagens,2, figsize=(5, 10))
    for i, depth in enumerate(range(numImagens-1, -1, -1)):
        imgBin = np.zeros((num_rows, num_cols), dtype=np.uint8)
        contour_img = np.zeros((num_rows, num_cols), dtype=np.uint8)
            
        for node in nodes_by_depth[depth]:
            for p in countors[node.id]:
                row = p // num_cols
                col = p % num_cols
                contour_img[row, col] = 1  

            for p in node.pixelsOfCC():
                row = p // num_cols
                col = p % num_cols
                imgBin[row, col] = 1  

        axes[i,0].imshow(imgBin, cmap='gray_r', vmax=1, vmin=0, interpolation='nearest')
        axes[i,1].imshow(contour_img, cmap='Reds')
        axes[i,0].set_title(f'Depth: {depth}')
        axes[i,0].axis('off')
       
    plt.show()

In [5]:
import numpy as np

def interpolate_image(img):
    H, W = img.shape
    H2, W2 = 2 * H - 1, 2 * W - 1

    out = np.empty((H2, W2), dtype=object)

    # Passo 1: copiar os valores originais (em pares)
    for i in range(H):
        for j in range(W):
            out[2*i, 2*j] = (img[i, j], img[i, j])  # intervalo degenerado

    # Passo 2: interpolação horizontal
    for i in range(H):
        for j in range(W - 1):
            a, b = img[i, j], img[i, j+1]
            out[2*i, 2*j+1] = (min(a, b), max(a, b))

    # Passo 3: interpolação vertical
    for i in range(H - 1):
        for j in range(W):
            a, b = img[i, j], img[i+1, j]
            out[2*i+1, 2*j] = (min(a, b), max(a, b))

    # Passo 4: interpolação central (saddle points)
    for i in range(H - 1):
        for j in range(W - 1):
            a = img[i, j]
            b = img[i, j+1]
            c = img[i+1, j]
            d = img[i+1, j+1]
            values = [a, b, c, d]
            out[2*i+1, 2*j+1] = (min(values), max(values))

    return out

img = np.array([
[3, 3],
[3, 0]
], dtype=np.uint8)

result = interpolate_image(img)

for row in result:
    print(" ".join(f"{x}" for x in row))

(3, 3) (3, 3) (3, 3)
(3, 3) (0, 3) (0, 3)
(3, 3) (0, 3) (0, 0)


In [7]:
import numpy as np

def interpolate_image_4c8c(img):
    numRows, numCols = img.shape
    interpRows = numRows * 2 + 1
    interpCols = numCols * 2 + 1

    min_img = np.full((interpRows, interpCols), np.inf, dtype=float)
    max_img = np.full((interpRows, interpCols), -np.inf, dtype=float)

    # Etapa 1: preenche os valores originais em posições ímpares
    for i in range(numRows):
        for j in range(numCols):
            row = 2 * i + 1
            col = 2 * j + 1
            val = img[i, j]
            min_img[row, col] = max_img[row, col] = val

    # Tabelas de vizinhos
    adjCircle = [(-1, -1), (-1, +1), (+1, -1), (+1, +1)]
    adjVert   = [(-1, 0), (+1, 0)]
    adjHorz   = [(0, -1), (0, +1)]

    # Etapa 2: interpolar os outros pixels
    for row in range(interpRows):
        for col in range(interpCols):
            if row % 2 == 1 and col % 2 == 1:
                continue  # valor original

            if row % 2 == 0 and col % 2 == 0:
                adj = adjCircle
            elif row % 2 == 0:
                adj = adjVert
            elif col % 2 == 0:
                adj = adjHorz
            

            vals = []
            for dr, dc in adj:
                r, c = row + dr, col + dc
                if 0 <= r < interpRows and 0 <= c < interpCols:
                    v_min = min_img[r, c]
                    v_max = max_img[r, c]
                    if v_min != np.inf and v_max != -np.inf:
                        vals.append((v_min, v_max))

            if vals:
                mins, maxs = zip(*vals)
                min_img[row, col] = min(mins)
                max_img[row, col] = max(maxs)

    return min_img.astype(int), max_img.astype(int)

img = np.array([
    [0, 8],
    [9, 4]
], dtype=np.uint8)

min_interp, max_interp = interpolate_image_4c8c(img)

for row_min, row_max in zip(min_interp, max_interp):
    print(" ".join(f"[{mn},{mx}]" for mn, mx in zip(row_min, row_max)))

[0,0] [0,0] [0,8] [8,8] [8,8]
[0,0] [0,0] [0,8] [8,8] [8,8]
[0,9] [0,9] [0,9] [4,8] [4,8]
[9,9] [9,9] [4,9] [4,4] [4,4]
[9,9] [9,9] [4,9] [4,4] [4,4]


In [11]:
imgInput = np.array([
    [5, 5, 5, 5, 5, 5, 5, 9, 9],
    [5, 5, 0, 0, 5, 5, 9, 5, 5],
    [5, 0, 5, 0, 5, 5, 9, 9, 5],
    [5, 0, 0, 5, 5, 5, 9, 9, 5],
    [5, 0, 0, 5, 5, 5, 5, 5, 5]
])
(num_rows, num_cols) = imgInput.shape

tos = mmcfilters.MorphologicalTree(imgInput, "4c8c")
print("#Nodes:", tos.numNodes)
nodes_by_depth = defaultdict(list)
dfs(tos.root)
print("Image:\n", imgInput)
printTree(tos.root)
print("\n", mapIDs(tos))

#Nodes: 5
Image:
 [[5 5 5 5 5 5 5 9 9]
 [5 5 0 0 5 5 9 5 5]
 [5 0 5 0 5 5 9 9 5]
 [5 0 0 5 5 5 9 9 5]
 [5 0 0 5 5 5 5 5 5]]
        id:0 
  ┌──────┼──────┐
 id:1   id:2   id:3 
                |   
               id:4 

 [[0 0 0 0 0 0 0 2 2]
 [0 0 3 3 0 0 1 0 0]
 [0 3 4 3 0 0 1 1 0]
 [0 3 3 0 0 0 1 1 0]
 [0 3 3 0 0 0 0 0 0]]


In [13]:
imgInput = np.array([
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],    
        [2, 1, 1, 1, 1, 7, 1, 7, 7, 4],
        [2, 1, 1, 1, 1, 1, 7, 7, 7, 4],
        [2, 1, 1, 1, 1, 2, 2, 7, 7, 4],
        [2, 1, 1, 1, 2, 2, 2, 7, 7, 4],
        [2, 2, 2, 1, 1, 7, 2, 7, 7, 4],
        [2, 1, 1, 2, 1, 1, 7, 7, 7, 4],
        [2, 1, 1, 2, 1, 1, 7, 7, 7, 4],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
])
(num_rows, num_cols) = imgInput.shape


tos = mmcfilters.MorphologicalTree(imgInput, "4c8c")
print("#Nodes:", tos.numNodes)
nodes_by_depth = defaultdict(list)
dfs(tos.root)
print(imgInput)
printTree(tos.root)
print("\n", mapIDs(tos))

#Nodes: 6
[[2 2 2 2 2 2 2 2 2 2]
 [2 1 1 1 1 7 1 7 7 4]
 [2 1 1 1 1 1 7 7 7 4]
 [2 1 1 1 1 2 2 7 7 4]
 [2 1 1 1 2 2 2 7 7 4]
 [2 2 2 1 1 7 2 7 7 4]
 [2 1 1 2 1 1 7 7 7 4]
 [2 1 1 2 1 1 7 7 7 4]
 [2 2 2 2 2 2 2 2 2 2]]
           id:0 
  ┌──────┬──┴───┬──────┐
 id:1   id:2   id:3   id:5 
         |                 
        id:4 

 [[0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 3 1 4 4 2]
 [0 1 1 1 1 1 4 4 4 2]
 [0 1 1 1 1 0 0 4 4 2]
 [0 1 1 1 0 0 0 4 4 2]
 [0 0 0 1 1 5 0 4 4 2]
 [0 1 1 0 1 1 4 4 4 2]
 [0 1 1 0 1 1 4 4 4 2]
 [0 0 0 0 0 0 0 0 0 0]]


In [35]:
imgInput = np.array([
    [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
    [10, 10,215,255,255,150,150, 10, 10, 10, 10, 10, 10, 10, 10],
    [10,150,255,255,255,255,255,150, 10, 10, 10, 10, 10, 10, 10],
    [10,215,225,150,225,255,255,255,255,150,150,150, 10, 10, 10],
    [10,225,225,150,150,215,255,255,255,255,255,255,150, 10, 10],
    [10,225,255,150, 10, 10,150,255,255,255,255,255,150, 10, 10],
    [10,215,255,150, 10, 10, 10,150,225,255,255,255,255,150,10],
    [10,150,255,255, 10, 10, 10,150,225,255,255,255,255,215,10],
    [10,150,255,255,150, 10, 10,150,225,150,150,150,255,150,10],
    [10,150,255,255,150, 10, 10,215,215, 10, 10, 10,150, 10,10],
    [10,150,255,255,150, 10,150,215,150, 10, 10, 10, 10, 10,10],
    [10, 10,150,255,225,150,225,215,150, 10, 10, 10, 10, 10,10],
    [10, 10,150,255,255,255,225,150, 10, 10, 10, 10, 10, 10,10],
    [10, 10,150,255,255,255,150, 10, 10, 10, 10, 10, 10, 10,10],
    [10, 10, 10,150,225,255,150, 10, 10, 10, 10, 10, 10, 10,10],
    [10, 10, 10, 10,150,150, 10, 10, 10, 10, 10, 10, 10, 10,10],
    [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,10]
], dtype=np.uint8)
(num_rows, num_cols) = imgInput.shape
#imgInput = 255 - imgInput

tos = mmcfilters.MorphologicalTree(imgInput, num_rows, num_cols)
print("#Nodes:", tos.numNodes)
nodes_by_depth = defaultdict(list)
dfs(tos.root)
print(imgInput)
printTree(tos.root)
print("\n", mapIDs(tos))

nodes = tos.listNodes

#Nodes: 5
[[ 10  10  10  10  10  10  10  10  10  10  10  10  10  10  10]
 [ 10  10 215 255 255 150 150  10  10  10  10  10  10  10  10]
 [ 10 150 255 255 255 255 255 150  10  10  10  10  10  10  10]
 [ 10 215 225 150 225 255 255 255 255 150 150 150  10  10  10]
 [ 10 225 225 150 150 215 255 255 255 255 255 255 150  10  10]
 [ 10 225 255 150  10  10 150 255 255 255 255 255 150  10  10]
 [ 10 215 255 150  10  10  10 150 225 255 255 255 255 150  10]
 [ 10 150 255 255  10  10  10 150 225 255 255 255 255 215  10]
 [ 10 150 255 255 150  10  10 150 225 150 150 150 255 150  10]
 [ 10 150 255 255 150  10  10 215 215  10  10  10 150  10  10]
 [ 10 150 255 255 150  10 150 215 150  10  10  10  10  10  10]
 [ 10  10 150 255 225 150 225 215 150  10  10  10  10  10  10]
 [ 10  10 150 255 255 255 225 150  10  10  10  10  10  10  10]
 [ 10  10 150 255 255 255 150  10  10  10  10  10  10  10  10]
 [ 10  10  10 150 225 255 150  10  10  10  10  10  10  10  10]
 [ 10  10  10  10 150 150  10  10  10  10  10

In [37]:
imgInput = np.array([
        [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14],
        [14, 14,213,253,241,193,122, 60, 14, 14, 14, 14, 14, 14, 14],
        [14,159,235,248,253,253,231,164, 91, 47, 49, 14, 14, 14, 14],
        [14,215,224,196,221,239,250,242,204,164,175,181, 85, 14, 14],
        [14,239,226,136,159,207,235,250,255,253,253,233,117, 14, 14],
        [14,226,242,151, 14, 14,174,230,251,255,254,249,184, 81, 14],
        [14,211,245,175, 14, 14, 14, 159,225,245,247,253,242,169,14],
        [14,183,250,210, 14, 14, 14, 130,215,233,220,230,236,212,14],
        [14,148,248,243,136, 14, 14, 153,243,212,164,195,220,168,14],
        [14,130,235,250,163, 14, 93, 212,224,136, 87, 96,122, 99,14],
        [14,125,224,251,180, 96,169, 232,169, 60, 32, 53, 72, 14,14],
        [14, 80,196,247,221,163,221, 207,108, 14, 12, 14, 14, 14,14],
        [14, 44,163,245,248,237,229, 136, 36, 14, 14, 14, 14, 14,14],
        [14, 40,126,231,243,241,190,  78, 14, 16, 14, 14, 14, 14,14],
        [14, 14, 71,175,224,214,127,  14, 14, 14, 14, 14, 14, 14,14],
        [14, 14, 29, 72,132,147, 59,  14, 14, 14, 14, 14, 14, 14,14],
        [14, 14, 14, 14, 14, 14, 14,  14, 14, 14, 14, 14, 14, 14,14]
])
(num_rows, num_cols) = imgInput.shape

tos = mmcfilters.MorphologicalTree(imgInput)
print("#Nodes:", tos.numNodes)
nodes_by_depth = defaultdict(list)
dfs(tos.root)
print(imgInput)
printTree(tos.root)
print("\n", mapIDs(tos))
nodes = tos.listNodes

#Nodes: 116
[[ 14  14  14  14  14  14  14  14  14  14  14  14  14  14  14]
 [ 14  14 213 253 241 193 122  60  14  14  14  14  14  14  14]
 [ 14 159 235 248 253 253 231 164  91  47  49  14  14  14  14]
 [ 14 215 224 196 221 239 250 242 204 164 175 181  85  14  14]
 [ 14 239 226 136 159 207 235 250 255 253 253 233 117  14  14]
 [ 14 226 242 151  14  14 174 230 251 255 254 249 184  81  14]
 [ 14 211 245 175  14  14  14 159 225 245 247 253 242 169  14]
 [ 14 183 250 210  14  14  14 130 215 233 220 230 236 212  14]
 [ 14 148 248 243 136  14  14 153 243 212 164 195 220 168  14]
 [ 14 130 235 250 163  14  93 212 224 136  87  96 122  99  14]
 [ 14 125 224 251 180  96 169 232 169  60  32  53  72  14  14]
 [ 14  80 196 247 221 163 221 207 108  14  12  14  14  14  14]
 [ 14  44 163 245 248 237 229 136  36  14  14  14  14  14  14]
 [ 14  40 126 231 243 241 190  78  14  16  14  14  14  14  14]
 [ 14  14  71 175 224 214 127  14  14  14  14  14  14  14  14]
 [ 14  14  29  72 132 147  59  14  14  14  